In [14]:
import requests
import zipfile
import io
import pandas as pd

# Download do arquivo ZIP em memória
response = requests.get("https://cdn.tse.jus.br/estatistica/sead/eleicoes/eleicoes2022/correspefet/CEFT_2t_RN_311020221100.zip" )
if response.status_code == 200:
    with zipfile.ZipFile(io.BytesIO(response.content)) as z:
        csv_file = [file for file in z.namelist() if file.endswith(".csv")][0]
        with z.open(csv_file) as f:
            df = pd.read_csv(f, encoding="ISO-8859-1", sep=";")
else:
    print(f"Falha ao baixar o arquivo. Código de status: {response.status_code}")


In [16]:
df

,DT_GERACAO,HH_GERACAO,ANO_ELEICAO,CD_PLEITO,DT_PLEITO,SG_UF,CD_MUNICIPIO,NM_MUNICIPIO,NR_ZONA,NR_SECAO,...,CD_CARGA_URNA_EFETIVADA,CD_CARGA_1_URNA_EFETIVADA,CD_CARGA_2_URNA_EFETIVADA,CD_FLASHCARD_URNA_EFETIVADA,DT_CARGA_URNA_EFETIVADA,DT_RECO_URNA_EFETIVADA,CD_ORIGEM_VOTO,DS_ORIGEM_VOTO,DS_DIVERGENCIA,NR_LOCAL_VOTACAO
0,31/10/2022,11:01:03,2022,407,30/10/2022,RN,16004,TENENTE LAURENTINO CRUZ,21,16,...,444569170192552795523074,444.569.170.192.552.795.,523.074,931D48EE,20/09/2022 18:14,20/09/2022 18:29,U,Urna Eletrônica,#NULO#,1015
1,31/10/2022,11:01:03,2022,407,30/10/2022,RN,16004,TENENTE LAURENTINO CRUZ,21,17,...,181943028166273422629247,181.943.028.166.273.422.,629.247,931D48EE,20/09/2022 16:43,20/09/2022 18:29,U,Urna Eletrônica,#NULO#,1015
2,31/10/2022,11:01:03,2022,407,30/10/2022,RN,16004,TENENTE LAURENTINO CRUZ,21,18,...,524777525572328425459658,524.777.525.572.328.425.,459.658,BC806D0A,21/09/2022 10:47,21/09/2022 11:01,U,Urna Eletrônica,#NULO#,1015
3,31/10/2022,11:01:03,2022,407,30/10/2022,RN,16004,TENENTE LAURENTINO CRUZ,21,44,...,596084319607911326086789,596.084.319.607.911.326.,86.789,931D48EE,20/09/2022 16:55,20/09/2022 18:29,U,Urna Eletrônica,#NULO#,1015
4,31/10/2022,11:01:03,2022,407,30/10/2022,RN,16004,TENENTE LAURENTINO CRUZ,21,47,...,353660402883987003028961,353.660.402.883.987.003.,28.961,931D48EE,20/09/2022 17:00,20/09/2022 18:29,U,Urna Eletrônica,#NULO#,1015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7669,31/10/2022,11:01:03,2022,407,30/10/2022,RN,30031,BARAÚNA,58,123,...,58028610203971061427867,058.028.610.203.971.061.,427.867,FD314358,01/10/2022 15:08,01/10/2022 15:25,U,Urna Eletrônica,#NULO#,1163
7670,31/10/2022,11:01:03,2022,407,30/10/2022,RN,30031,BARAÚNA,58,137,...,675490677818508820501145,675.490.677.818.508.820.,501.145,32DC31D0,19/09/2022 19:04,19/09/2022 19:09,U,Urna Eletrônica,**,1163
7671,31/10/2022,11:01:03,2022,407,30/10/2022,RN,30031,BARAÚNA,58,139,...,37826322001532514622924,037.826.322.001.532.514.,622.924,FD314358,20/09/2022 10:44,20/09/2022 11:23,U,Urna Eletrônica,#NULO#,1180
7672,31/10/2022,11:01:03,2022,407,30/10/2022,RN,30031,BARAÚNA,58,140,...,351058721133739428544293,351.058.721.133.739.428.,544.293,FD314358,20/09/2022 10:51,20/09/2022 11:23,U,Urna Eletrônica,#NULO#,1066
